## [네이버 배구 경기 정보](https://sports.news.naver.com/volleyball/schedule/index)

- 소스보기에 데이터가 없다. -> ajax, iframe, 웹소켓
- 우클릭 메뉴에 프레임 소스보기가 없다. -> ajax, 웹소켓
- 크롬 개발자 도구의 network 탭에 xhr로 표시된다 -> ajax
- ajax 통신 주소를 통해 requets로 요청하고, bs4로 분석

In [1]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, fileName) :
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        '경기 날짜' : [],
        '팀1 이름' : [],
        '팀2 이름' : [],
        '팀1 세트 스코어' : [],
        '팀2 세트 스코어' : []
    }
    
    # 현재 날짜
    a1 = source.select_one('body > div > ul > li.on > a')
    # print(a1)
    a2 = a1.attrs['onclick']
    a3 = a2.split("'")
    data1 = a3[1]
    print(data1)
    
    # 왼쪽경기
    a1 = source.select_one('#todaySchedule0 > div.inner_lft.end')
    # print(a1)
    if a1 != None :
        # 경기 날짜
        data_dict['경기 날짜'].append(data1)
        
        # 팀1 이름
        a2 = a1.select_one('div.vs_lft > strong.vs_lft_name')
        data2 = a2.text.strip()
        data_dict['팀1 이름'].append(data2)
        
        # 팀2 이름
        a2 = a1.select_one('div.vs_rgt > strong.vs_rgt_name')
        data2 = a2.text.strip()
        data_dict['팀2 이름'].append(data2)
        
        # 팀1 세트 스코어
        a2 = a1.select_one('div.vs_lft > strong.vs_num')
        data2 = a2.text.strip()
        data_dict['팀1 세트 스코어'].append(data2)
        
        # 팀2 세트 스코어
        a2 = a1.select_one('div.vs_rgt > strong.vs_num')
        data2 = a2.text.strip()
        data_dict['팀2 세트 스코어'].append(data2)
        
    # 오른쪽 경기
    a1 = source.select_one('#todaySchedule0 > div.inner_rgt.end')
    # print(a1)
    if a1 != None :
        # 경기 날짜
        data_dict['경기 날짜'].append(data1)
        
        # 팀1 이름
        a2 = a1.select_one('div.vs_lft > strong.vs_lft_name')
        data2 = a2.text.strip()
        data_dict['팀1 이름'].append(data2)
        
        # 팀2 이름
        a2 = a1.select_one('div.vs_rgt > strong.vs_rgt_name')
        data2 = a2.text.strip()
        data_dict['팀2 이름'].append(data2)
        
        # 팀1 세트 스코어
        a2 = a1.select_one('div.vs_lft > strong.vs_num')
        data2 = a2.text.strip()
        data_dict['팀1 세트 스코어'].append(data2)
        
        # 팀2 세트 스코어
        a2 = a1.select_one('div.vs_rgt > strong.vs_num')
        data2 = a2.text.strip()
        data_dict['팀2 세트 스코어'].append(data2)
        
    print(data_dict)
    
    # 경기가 있는 날만 저장한다.
    if len(data_dict['경기 날짜']) > 0 :
        # 데이터 프레임을 생성한다.
        df1 = pd.DataFrame(data_dict)
        # display(df1)

        # 저장한다.
        # 파일이 없다면 : 첫 번째 저장
        if os.path.exists(fileName) == False :
            df1.to_csv(fileName, encoding='utf-8-sig', index=False)
        # 파일이 있다면..
        else :
            df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [4]:
# 다음 날짜를 가져오는 함수
def getNext(source) :
    
    a1 = source.select_one('body > div > ul > li.on + li> a')
    a2 = a1.attrs['onclick']
    a3 = a2.split("'")
    data1 = a3[1]
    
    return data1

In [5]:
## 수집 날짜
today = '20130101'
fileName = 'kovo_info.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)
    
while True :
    time.sleep(1)
    clear_output(wait=True)

    # 요청 주소
    site = f'https://sports.news.naver.com/schedule/scoreBoard?date={today}&uCategory=kvolleyball'

    # 요청한다.
    a1 = getSource(site)
    source = bs4.BeautifulSoup(a1)

    # 데이터를 수집한다
    getData(source, fileName)

    # 다음 날짜를 가져온다.
    today = getNext(source)
    # print(today)
    
    # 2022년이 되면 중단시킨다.
    if today.startswith('2022') :
        break
        
print('수집 완료')

20130102
{'경기 날짜': ['20130102', '20130102'], '팀1 이름': ['IBK기업은행', '드림식스'], '팀2 이름': ['현대건설', 'KEPCO'], '팀1 세트 스코어': ['3', '3'], '팀2 세트 스코어': ['0', '0']}


KeyboardInterrupt: 